# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846709088067                   -0.70    4.5         
  2   -7.852299650873       -2.25       -1.53    1.0   16.8ms
  3   -7.852612736496       -3.50       -2.55    1.5   18.5ms
  4   -7.852645959018       -4.48       -2.87    2.8   23.4ms
  5   -7.852646490607       -6.27       -3.16    1.0   17.0ms
  6   -7.852646678361       -6.73       -3.98    1.0   17.1ms
  7   -7.852646686268       -8.10       -5.24    1.5   19.1ms
  8   -7.852646686725       -9.34       -5.55    2.5   23.4ms
  9   -7.852646686730      -11.34       -6.64    1.0   17.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846846558982                   -0.70           4.8         
  2   -7.852526297981       -2.25       -1.64   0.80    2.0    204ms
  3   -7.852635474106       -3.96       -2.73   0.80    1.0   15.7ms
  4   -7.852646461353       -4.96       -3.25   0.80    2.2   20.6ms
  5   -7.852646668297       -6.68       -4.06   0.80    1.0   15.7ms
  6   -7.852646686317       -7.74       -4.71   0.80    1.8   18.9ms
  7   -7.852646686723       -9.39       -5.40   0.80    1.8   19.2ms
  8   -7.852646686729      -11.24       -6.58   0.80    1.2   16.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.385061e+01     4.191001e+00
 * time: 0.045449018478393555
     1     1.405128e+00     1.896529e+00
 * time: 0.21868896484375
     2    -1.173156e+00     2.187357e+00
 * time: 0.23662400245666504
     3    -3.582531e+00     1.746044e+00
 * time: 0.2623000144958496
     4    -4.933845e+00     1.600797e+00
 * time: 0.28797292709350586
     5    -6.660919e+00     1.214703e+00
 * time: 0.31387901306152344
     6    -7.275316e+00     8.645262e-01
 * time: 0.3396749496459961
     7    -7.531566e+00     5.931361e-01
 * time: 0.3575730323791504
     8    -7.636056e+00     1.338154e-01
 * time: 0.37552595138549805
     9    -7.697256e+00     2.478907e-01
 * time: 0.39334797859191895
    10    -7.731793e+00     1.070115e-01
 * time: 0.41123199462890625
    11    -7.758378e+00     1.262656e-01
 * time: 0.42917299270629883
    12    -7.794975e+00     1.130548e-01
 * time: 0.44702601432800293
    13    -7.827166e+00     5.827463e-02
 * time: 0.46

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846739220598                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645860694                   -1.64         
  2   -7.852646686730       -6.08       -3.70    1.49s
  3   -7.852646686730      -13.22       -7.21    151ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.601276412181086e-7
|ρ_newton - ρ_scfv| = 4.948537675598166e-7
|ρ_newton - ρ_dm|   = 1.9298545733695725e-9
